In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

1. Define a function named get_lower_and_upper_bounds that has two arguments. The first argument is a pandas Series. The second argument is the multiplier, which should have a default argument of 1.5.

In [18]:
def get_lower_and_upper_bounds(Series, multiplier = 1.5):
    q1 = Series.quantile(.25)
    q3 = Series.quantile(.75)
    iqr = q3 - q1
    upper = q3 + (iqr*multiplier)
    lower = q1 - (iqr*multiplier)
    return lower, upper

Using lemonade.csv dataset and focusing on continuous variables:

1) Use the IQR Range Rule and the upper and lower bounds to identify the lower outliers of each column of lemonade.csv, using the multiplier of 1.5. Do these lower outliers make sense? Which outliers should be kept?

In [3]:
lemon = pd.read_csv('https://gist.githubusercontent.com/ryanorsinger/19bc7eccd6279661bd13307026628ace/raw/e4b5d6787015a4782f96cad6d1d62a8bdbac54c7/lemonade.csv')

In [5]:
lemon.head(3)

,Date,Day,Temperature,Rainfall,Flyers,Price,Sales
0,1/1/17,Sunday,27.0,2.00,15,0.5,10
1,1/2/17,Monday,28.9,1.33,15,0.5,13
2,1/3/17,Tuesday,34.5,1.33,27,0.5,15


In [21]:
lemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         365 non-null    object 
 1   Day          365 non-null    object 
 2   Temperature  365 non-null    float64
 3   Rainfall     365 non-null    float64
 4   Flyers       365 non-null    int64  
 5   Price        365 non-null    float64
 6   Sales        365 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 20.1+ KB


In [16]:
lemon.iloc[:,2:]

,Temperature,Rainfall,Flyers,Price,Sales
0,27.0,2.00,15,0.5,10
1,28.9,1.33,15,0.5,13
2,34.5,1.33,27,0.5,15
3,44.1,1.05,28,0.5,17
4,42.4,1.00,33,0.5,18
...,...,...,...,...,...
360,42.7,1.00,33,0.5,19
361,37.8,1.25,32,0.5,16
362,39.5,1.25,17,0.5,15
363,30.9,1.43,22,0.5,13


In [20]:
for col in lemon.iloc[:,2:]:
    lower, upper = get_lower_and_upper_bounds(lemon[col], 1.5)
    print(f'Ranges for {col}')
    print()
    print(f'Lower: {lower}')
    print(f'Upper: {upper}')
    print('-----------------')

Ranges for Temperature

Lower: 16.700000000000003
Upper: 104.7
-----------------
Ranges for Rainfall

Lower: 0.26
Upper: 1.3
-----------------
Ranges for Flyers

Lower: 4.0
Upper: 76.0
-----------------
Ranges for Price

Lower: 0.5
Upper: 0.5
-----------------
Ranges for Sales

Lower: 5.0
Upper: 45.0
-----------------


**Temperature**: In this day and age of climate change, I wouldn't even know where to begin to determine what would be appropriate bounds for temperature.

**Rainfall**: Yes, these bounds look reasonable to me, assuming the unit is inches.

**Flyers**: Yes, if this is amount of flyers posted (and it means it's targeting effectiveness of advertising).

**Price**: Doesn't seem like price changed, so yes, it's reasonable bounds.

**Sales**: If price of lemonade is only $0.50, then yes, these numbers are reasonable.

2) Use the IQR Range Rule and the upper and upper bounds to identify the upper outliers of each column of lemonade.csv, using the multiplier of 1.5. Do these upper outliers make sense? Which outliers should be kept?

3) Using the multiplier of 3, IQR Range Rule, and the lower bounds, identify the outliers below the lower bound in each colum of lemonade.csv. Do these lower outliers make sense? Which outliers should be kept?

4) Using the multiplier of 3, IQR Range Rule, and the upper bounds, identify the outliers above the upper_bound in each colum of lemonade.csv. Do these upper outliers make sense? Which outliers should be kept?